In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Adapted from the Google Gemini "cookbook" found here: https://github.com/google-gemini/cookbook/tree/main

In [30]:
# %pip install -U -q 'google-genai'


In [29]:
with open('../gemini.key', 'r') as file:
    GOOGLE_API_KEY = file.read()[:-1]



In [23]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

In [24]:
model_name = "gemini-2.0-flash" # @param ["gemini-1.5-flash-latest","gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.0-pro-exp-02-05"] {"allow-input":true, isTemplate: true}

### Get sample videos

In [9]:
# Load sample images
# !wget https://storage.googleapis.com/generativeai-downloads/videos/Pottery.mp4 -O Pottery.mp4 -q
# !wget https://storage.googleapis.com/generativeai-downloads/videos/Jukin_Trailcam_Videounderstanding.mp4 -O Trailcam.mp4 -q
# !wget https://storage.googleapis.com/generativeai-downloads/videos/post_its.mp4 -O Post_its.mp4 -q
# !wget https://storage.googleapis.com/generativeai-downloads/videos/user_study.mp4 -O User_study.mp4 -q

In [25]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(file=video_file_name)

  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name)

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + video_file.uri)

  return video_file

#pottery_video = upload_video('Pottery.mp4')
trailcam_video = upload_video('Trailcam.mp4')
post_its_video = upload_video('Post_its.mp4')
user_study_video = upload_video('User_study.mp4')

Waiting for video to be processed.
Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/6amx0mm1k0an
Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/jixi6940mzfs
Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/e3zz14tvhw2d


### Imports

In [27]:
import json
from PIL import Image
from IPython.display import display, Markdown, HTML

In [28]:
prompt = "For each scene in this video, generate captions that describe the scene along with any spoken text placed in quotation marks. Place each caption into an object with the timecode of the caption in the video."  # @param ["For each scene in this video, generate captions that describe the scene along with any spoken text placed in quotation marks. Place each caption into an object with the timecode of the caption in the video.", "Organize all scenes from this video in a table, along with timecode, a short description, a list of objects visible in the scene (with representative emojis) and an estimation of the level of excitement on a scale of 1 to 10"] {"allow-input":true}

video = trailcam_video # @param ["trailcam_video", "pottery_video", "post_its_video", "user_study_video"] {"type":"raw","allow-input":true}

response = client.models.generate_content(
    model=model_name,
    contents=[
        video,
        prompt,
    ]
)

Markdown(response.text)

```json
[
  {
    "00:00": "A camera placed in the woods shows a gray fox walk toward the camera and then away from it. As it walks off, a second fox comes into view and approaches the first fox. Then one climbs on a rock."
  },
  {
    "00:16": "The video transitions to black and white and shows a mountain lion walking in the woods looking for something on the ground. At [00:00:28] it looks up and shakes its head as though it heard or sensed something."
  },
  {
    "00:35": "The video cuts to black and white and shows two foxes wandering around in the woods. One is trying to dig something out of the ground. At [00:00:49] the fox succeeds and the other fox grabs it, and the action becomes blurred because the foxes fight with each other. Eventually the action stops and you can see the two foxes on top of some rocks. One then wanders off and the other stays."
  },
  {
    "01:05": "The scene is now in black and white and shows a mountain lion walk in to the left of the camera. After it has been on the scene for a little while, it walks down to the left of the camera and another mountain lion walks into view on top of a rock. The lion on the rock walks down to the left of the camera and joins the other lion."
  },
  {
    "01:22": "The mountain lion is followed and after a few moments it comes into view and walks directly toward the camera. As it gets closer, it is difficult to see but eventually it walks past and to the left of the camera."
  },
  {
    "01:28": "A bobcat appears and walks towards the camera, stopping for a moment and sniffing the ground. The bobcat looks up, then continues to look around the woods and sniff the ground."
  },
  {
    "01:51": "The scene now features color video and shows a brown bear walk towards the camera and partially block the view of the camera. It then turns around and walks to the right of the camera. As it does that, you can see another brown bear following it."
  },
  {
    "01:57": "The scene again turns black and white and shows a mountain lion walking toward the right of the camera."
  },
  {
    "02:04": "Again the camera gets too close to the animal. The animal walks away from the camera. After a little while it comes into view again and smells around the ground."
  },
  {
    "02:23": "The camera is now set up on a hill overlooking a city. It is nighttime and you can see the city lights. A gray fox comes into view and looks around a little. It smells around a little and then sits up on its haunches and stares at the city."
  },
  {
    "02:34": "A bear walks into view and appears to attempt to sniff something on the ground. The animal leaves view. \nThen, a mountain lion walks toward the camera, then away. "
  },
  {
    "02:52": "A mountain lion wanders around and smells the ground."
  },
  {
    "03:05": "Two black bear cubs and their mother walk around and look for food. One of them stops and cleans itself."
  },
  {
    "04:22": "The camera is in the woods and it is black and white, and you can see a bobcat in front of the camera. The bobcat looks around, then jumps over the log and looks directly at the camera. The bobcat comes down from the log and looks directly at the camera. It then goes off to the right of the screen and stops and looks around again before exiting."
  },
  {
    "04:57": "A mountain lion wanders around and sniffs the ground."
  }
]
```